# <B>Earthquake Events - Gold layer

In [ ]:
from pyspark.sql.functions import when, col,udf
from pyspark.sql.types import StringType

import reverse_geocoder as rg


#### <b>Reading silver table into the gold layer for further processing.
1. I have filter the table to fetch only the new so that we are not processing duplicate values.

In [ ]:
df=spark.read.table("earthquake_sliver").filter(col(time)>start_date)

#### <b>Created a function to get country code by using revers_geocoder instance rg

In [ ]:
def get_country_code(lat,lon):
    coordinate=(float(lat),float(lon))
    return rg.search(coordinate)[0].get('cc')

#### <b>Registering above function(udf) so that we can use it in spark

In [ ]:
#Registering the udfs so that we can use this function in spark dataframe
get_country_code_udf=udf(get_country_code,StringType())

#### <B>Creating new field with name "Countery_code" by running registerd(udf) function

In [ ]:
df_with_location = df.withColumn("countery_code",get_country_code_udf(col('latitude'),col('longituede')))

In [ ]:
display(df_with_location)


#### <b>Creating bucket for the earthquake magnitude strength  

In [ ]:
df_with_location_sig = df_with_location.withColumn('sig_class',
when(col("strenth_of_earthquake")<100,"Low").\
when((col("strenth_of_earthquake")>=100) & (col("strenth_of_earthquake")<500),"Moderate").\
otherwise("High")
)

In [ ]:
display(df_with_location_sig)

#### <B>Saving the final dataframe as Gold Table

In [ ]:
## Wirting the final processed table to gold layer
df_with_location_sig.write.mode("append").saveAsTable("earthquake_gold")